In [ ]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
nltk.download('sentiwordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
# 영어 댓글 데이터 전처리
data = pd.read_csv('/content/drive/Shareddrives/데청캠/웹툰 세계화/데이터 모음집/ew_2_comment.csv')

#정규화
data['adj_comment'] = data['best_comment'].str.replace("[^0-9a-zA-Z ]","").str.lower().str.replace('^ +', "")

#앞에 TOP이라는 글씨 쓰인것들 제거
for i in range(len(data['adj_comment'])):
  if data['adj_comment'][i][:3] == 'top':
    data['adj_comment'][i] = data['adj_comment'][i][3:]

In [ ]:
# 형태소 분석
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB

In [ ]:
#sentiwordnet 감성분석
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화 
    pos_sentiment = 0.0
    neg_sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag) # 품사 태깅
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue                   
            lemma = lemmatizer.lemmatize(word, pos=wn_tag) # 어근 추출
            if not lemma:
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            synset = synsets[0] 
            swn_synset = swn.senti_synset(synset.name()) # 사전속 해당 synset
            pos_sentiment+= swn_synset.pos_score()
            neg_sentiment+=swn_synset.neg_score()      
            tokens_count += 1
    
    if not tokens_count:
        return (0,0)
    return (pos_sentiment,neg_sentiment)

In [ ]:
score = []
for i in data['best_comment']:
    score.append(swn_polarity(i))
total_data = pd.concat([data,result_score],axis=1)